In [1]:
import pandas as pd
from pathlib import Path

# Ścieżka do pliku JSON
json_file = Path("../data/all_offers.json")


# Wczytanie danych do DataFrame
df = pd.read_json(json_file)

# Usunięcie kolumny 'zdjecia'
df = df.drop(columns=["zdjecia"], errors="ignore")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14679 entries, 0 to 14678
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 14679 non-null  int64  
 1   marka              14679 non-null  object 
 2   model              14679 non-null  object 
 3   rok_produkcji      14679 non-null  int64  
 4   przebieg           14678 non-null  object 
 5   rodzaj_paliwa      14679 non-null  object 
 6   pojemnosc_silnika  14246 non-null  object 
 7   moc_silnika        14665 non-null  object 
 8   skrzynia_biegow    14676 non-null  object 
 9   naped              13729 non-null  object 
 10  liczba_drzwi       14629 non-null  float64
 11  typ_nadwozia       14679 non-null  object 
 12  kolor              14679 non-null  object 
 13  liczba_miejsc      13253 non-null  float64
 14  stan_pojazdu       14679 non-null  object 
 15  bezwypadkowosc     14679 non-null  bool   
 16  cena               146

In [3]:
df['czy_cesja'].sum()

np.int64(3007)

In [4]:
df['czy_uszkodzony'].sum()

np.int64(718)

In [5]:
df[(df['czy_cesja']) | (df['czy_uszkodzony'])].shape[0]

3720

In [6]:
df_cleaned = df[~((df['czy_cesja']) | (df['czy_uszkodzony']))]

In [7]:
df_cleaned['liczba_miejsc'] = df_cleaned['liczba_miejsc'].fillna(5)
df_cleaned['naped'] = df_cleaned['naped'].fillna("Na tylne koła")

C:\Users\tthaddey\AppData\Local\Temp\ipykernel_7216\3364323884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['liczba_miejsc'] = df_cleaned['liczba_miejsc'].fillna(5)
C:\Users\tthaddey\AppData\Local\Temp\ipykernel_7216\3364323884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['naped'] = df_cleaned['naped'].fillna("Na tylne koła")


In [8]:
df_cleaned = df_cleaned.dropna()

In [9]:
from currency_converter import CurrencyConverter
c = CurrencyConverter()

def convert_to_pln(row):
    if row['waluta'] != 'PLN':  # Sprawdź, czy waluta jest różna od PLN
        try:
            # Konwersja ceny do PLN
            return c.convert(row['cena'], row['waluta'], 'PLN')
        except ValueError as e:
            print(f"Błąd przy konwersji dla ID {row['id']}: {e}")
            return None  # Jeśli nie uda się przekonwertować
    return row['cena']  # Jeśli waluta to PLN, zwróć oryginalną cenę

df_cleaned['cena'] = df_cleaned.apply(convert_to_pln, axis=1)
df_cleaned['waluta'] = 'PLN'


In [10]:
df_cleaned = df_cleaned.loc[:, df_cleaned.nunique() > 1]

In [11]:
# Konwersja kolumn na wartości numeryczne
df_cleaned['przebieg'] = df_cleaned['przebieg'].str.replace(' km', '', regex=False).str.replace(' ', '').astype(float)
df_cleaned['pojemnosc_silnika'] = df_cleaned['pojemnosc_silnika'].str.replace(' cm3', '', regex=False).str.replace(' ', '').astype(float)
df_cleaned['moc_silnika'] = df_cleaned['moc_silnika'].str.replace(' KM', '', regex=False).str.replace(' ', '').astype(float)

In [12]:
df_cleaned.nunique()

id                   10714
model                   37
rok_produkcji           52
przebieg              4003
rodzaj_paliwa            5
pojemnosc_silnika       82
moc_silnika            166
skrzynia_biegow          2
naped                    5
liczba_drzwi             4
typ_nadwozia             9
kolor                   17
liczba_miejsc            7
stan_pojazdu             2
bezwypadkowosc           2
cena                  2285
czy_prywatny             2
lokalizacja           1832
url                  10714
dtype: int64

In [13]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10714 entries, 0 to 14677
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10714 non-null  int64  
 1   model              10714 non-null  object 
 2   rok_produkcji      10714 non-null  int64  
 3   przebieg           10714 non-null  float64
 4   rodzaj_paliwa      10714 non-null  object 
 5   pojemnosc_silnika  10714 non-null  float64
 6   moc_silnika        10714 non-null  float64
 7   skrzynia_biegow    10714 non-null  object 
 8   naped              10714 non-null  object 
 9   liczba_drzwi       10714 non-null  float64
 10  typ_nadwozia       10714 non-null  object 
 11  kolor              10714 non-null  object 
 12  liczba_miejsc      10714 non-null  float64
 13  stan_pojazdu       10714 non-null  object 
 14  bezwypadkowosc     10714 non-null  bool   
 15  cena               10714 non-null  float64
 16  czy_prywatny       10714 no

In [14]:
df_cleaned.head(10)

,id,model,rok_produkcji,przebieg,rodzaj_paliwa,pojemnosc_silnika,moc_silnika,skrzynia_biegow,naped,liczba_drzwi,typ_nadwozia,kolor,liczba_miejsc,stan_pojazdu,bezwypadkowosc,cena,czy_prywatny,lokalizacja,url
0,100002067157270,Seria 3,2005,364000.0,Diesel,1995.0,150.0,Automatyczna,Na tylne koła,5.0,Kombi,Srebrny,5.0,Używany,False,8600.0,True,Warszawa,https://www.otomoto.pl/osobowe/oferta/bmw-seri...
2,100016885962989,Seria 5,1999,240000.0,Benzyna,3498.0,245.0,Automatyczna,Na tylne koła,4.0,Sedan,Czarny,5.0,Używany,True,42700.0,True,Poznań,https://www.otomoto.pl/osobowe/oferta/bmw-seri...
4,100049777153183,Seria 6,2004,325000.0,Benzyna+LPG,4398.0,333.0,Automatyczna,Na tylne koła,2.0,Coupe,Szary,4.0,Używany,True,68000.0,True,Władysławowo,https://www.otomoto.pl/osobowe/oferta/bmw-seri...
7,100075118000516,X3,2019,76000.0,Diesel,1995.0,190.0,Automatyczna,4x4 (stały),5.0,SUV,Granatowy,5.0,Używany,True,182000.0,True,Węgrzce Wielkie,https://www.otomoto.pl/osobowe/oferta/bmw-x3-b...
8,100078009534233,Seria 3,2010,303000.0,Diesel,1995.0,183.0,Automatyczna,4x4 (dołączany automatycznie),4.0,SUV,Czarny,5.0,Używany,False,25700.0,True,Bielsko-Biała,https://www.otomoto.pl/osobowe/oferta/bmw-seri...
12,100103490336814,Seria 4,2018,51555.0,Benzyna,1998.0,252.0,Automatyczna,4x4 (dołączany automatycznie),5.0,Sedan,Biały,4.0,Używany,False,69000.0,False,Warszawa,https://www.otomoto.pl/osobowe/oferta/bmw-seri...
15,100133161987737,X5,2020,141649.0,Benzyna,2998.0,340.0,Automatyczna,4x4 (stały),5.0,SUV,Szary,5.0,Używany,True,219900.0,False,Swarzędz,https://www.otomoto.pl/osobowe/oferta/bmw-x5-b...
17,100152227273235,Seria 1,2018,68220.0,Diesel,1496.0,116.0,Manualna,Na przednie koła,5.0,Kompakt,Srebrny,5.0,Używany,True,57500.0,False,Sosnowiec,https://www.otomoto.pl/osobowe/oferta/bmw-seri...
18,100172667028716,X3,2017,160000.0,Diesel,1995.0,190.0,Automatyczna,4x4 (stały),5.0,SUV,Granatowy,5.0,Używany,False,114999.0,False,Radwanice,https://www.otomoto.pl/osobowe/oferta/bmw-x3-b...
19,100179132450337,Seria 5,2012,239000.0,Diesel,1995.0,218.0,Automatyczna,4x4 (stały),5.0,Kombi,Czarny,5.0,Używany,True,46600.0,True,Dębica,https://www.otomoto.pl/osobowe/oferta/bmw-seri...


In [15]:
from pathlib import Path
from sklearn.model_selection import train_test_split

output_file = Path("../data/clean_data.json")
#df_cleaned.to_json(output_file, orient='records', lines=True, force_ascii=False)
train_data, test_data = train_test_split(df_cleaned, test_size=0.2, random_state=42)
output_file = Path("../data/clean_data_train.json")
train_data.to_json(output_file, orient='records', lines=True, force_ascii=False)
output_file = Path("../data/clean_data_test.json")
test_data.to_json(output_file, orient='records', lines=True, force_ascii=False)